# Short Put
[Projection option](https://www.projectoption.com/selling-put-options/)
[Tastytrade Short Put check list](https://www.tastytrade.com/tt/shows/mike-and-his-whiteboard/episodes/trade-checklist-short-put-04-11-2016)
[Tastytrade Short Put Adjustments](https://www.tastytrade.com/tt/shows/mike-and-his-whiteboard/episodes/trading-strategy-3-short-put-adjustments-01-28-2016)

In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import pandas as pd
import matplotlib.pyplot as plt
from ib_insync import *
import os
import sys

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from optopus.ib_adapter import IBBrokerAdapter
from optopus.optopus import Optopus
from optopus.data_objects import UStock
from optopus.utils import pdo

%matplotlib inline


In [2]:
watch_list = [UStock('SPY'),
              UStock('QQQ'),
              UStock('EEM'),
              UStock('IWM'),
              UStock('FXI'),
              UStock('VXX')]

util.startLoop()
opt = Optopus(IBBrokerAdapter(IB(), host='127.0.0.1', port=7497, client=6), watch_list)
opt.start()

[Initializating managers]
[Connecting to IB broker]
[Updating portfolio]
[Adding underlyings]
- Creating underlyings: ......
- Retriving current data: ......
- Retriving historical data: ......
- Retriving historical IV data: ......
- Computing fields: ......
[Started]



## Implied Volatility
Check IV, IV Rank and IV Percentile

In [3]:
df = pdo(opt.underlyings(['market_price', 'volume', 'bid', 'ask', 'IV_h', 'IV_rank_h', 'IV_percentile_h', 'volume_h']))

In [4]:
df['spread'] = df['ask'] - df['bid']
df

,market_price,volume,bid,ask,IV_h,IV_rank_h,IV_percentile_h,volume_h,spread
code,,,,,,,,,
EEM,42.69,497579,42.69,42.70,0.180144,13.571453,79.365079,473848,0.01
FXI,42.08,264784,42.07,42.08,0.196907,31.110036,61.111111,228668,0.01
IWM,170.95,103340,170.94,170.95,0.112852,5.969195,14.285714,86784,0.01
QQQ,181.11,212124,181.11,181.12,0.133092,13.623246,35.317460,154100,0.01
SPY,285.93,312967,285.93,285.94,0.089627,9.354246,53.571429,275936,0.01
VXX,28.95,273960,28.95,28.96,0.656697,12.863917,62.698413,264492,0.01


In [5]:
df_candidates = df[(df['spread'] <= 0.03) & (df['volume_h'] > 1000) & (df['IV_percentile_h'] > 40)]
df_candidates = df_candidates.sort_values('IV_percentile_h', ascending=False)
df_candidates

,market_price,volume,bid,ask,IV_h,IV_rank_h,IV_percentile_h,volume_h,spread
code,,,,,,,,,
EEM,42.69,497579,42.69,42.70,0.180144,13.571453,79.365079,473848,0.01
VXX,28.95,273960,28.95,28.96,0.656697,12.863917,62.698413,264492,0.01
FXI,42.08,264784,42.07,42.08,0.196907,31.110036,61.111111,228668,0.01
SPY,285.93,312967,285.93,285.94,0.089627,9.354246,53.571429,275936,0.01


In [6]:
df_oc = pdo(opt.option_chain(UStock(df_candidates.iloc[0].name), ['option_price', 'delta', 'DTE']))

[Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=40.0, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=40.5, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=41.0, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=41.5, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=42.0, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=42.5, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=43.0, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=43.5, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='20180921', strike=44.0, right='P', exchange='SMART'), Option(symbol='EEM', lastTradeDateOrContractMonth='201

In [7]:
df_oc

,expiration,strike,right,option_price,delta,DTE
code,,,,,,
EEM,2018-09-21,40.0,P,0.161773,-0.124695,29
EEM,2018-09-21,40.5,P,0.219082,-0.164198,29
EEM,2018-09-21,41.0,P,0.301379,-0.216508,29
EEM,2018-09-21,41.5,P,0.411782,-0.280507,29
EEM,2018-09-21,42.0,P,0.559257,-0.357179,29
EEM,2018-09-21,42.5,P,0.748374,-0.443897,29
EEM,2018-09-21,43.0,P,0.985418,-0.537823,29
EEM,2018-09-21,43.5,P,1.268746,-0.632563,29
EEM,2018-09-21,44.0,P,1.600141,-0.721603,29


In [8]:
df_oc[(df_oc['right'] == 'P')]

,expiration,strike,right,option_price,delta,DTE
code,,,,,,
EEM,2018-09-21,40.0,P,0.161773,-0.124695,29
EEM,2018-09-21,40.5,P,0.219082,-0.164198,29
EEM,2018-09-21,41.0,P,0.301379,-0.216508,29
EEM,2018-09-21,41.5,P,0.411782,-0.280507,29
EEM,2018-09-21,42.0,P,0.559257,-0.357179,29
EEM,2018-09-21,42.5,P,0.748374,-0.443897,29
EEM,2018-09-21,43.0,P,0.985418,-0.537823,29
EEM,2018-09-21,43.5,P,1.268746,-0.632563,29
EEM,2018-09-21,44.0,P,1.600141,-0.721603,29
